In [ ]:
import json
import os
import pandas as pd
import requests
import time

from lxml import etree, html
from bs4 import BeautifulSoup

#headers for GET requests
headers = {'accept': "application/json", 'accept': "text/csv"}

#global variables for storing day and the data_array
data_array={} #this dictionary will store all the data


#the following function retrieves the data from bulleted list
#list_object - Body of the list item (text content)
def parse_list(list_object):
    if ("Hospitalized (Ever)" not in list_object and
        "Death" not in list_object and
        "Investigated Cases" not in list_object and
        "0 to 17" not in list_object and
        "18 to 40" not in list_object and
        "41 to 65" not in list_object and
        "over 65" not in list_object and
        "City of Los Angeles" not in list_object and
        "http" not in list_object and
        "Long Beach" not in list_object and
        "Hispanic" not in list_object and
        "White" not in list_object and
        "Black" not in list_object and
        "Other" not in list_object and 
        "Asian" not in list_object and 
        "Male" not in list_object and
        "Female" not in list_object and
        "Under Investigation" not in list_object
       ):
        if "\t" in list_object or "--" in list_object:
            if "\t" in list_object:
                out=list_object.split("\t")
            else:
                out=list_object.split("--")    
                out[0]=str(out[0]).replace("*","")
                out[0]=str(out[0]).replace("--","")
                out[1]=str(out[1]).replace("--","0")
                out[1]=str(out[1].replace("<",""))
                out[1]=str(out[1]).lstrip("0")
                out[1]=str(out[1]).replace("*","")
                if not out[1]:
                    out[1]="0"
        
            return out
    

In [9]:
def get_data(urlcomp, press_release_id):
    rcomp = requests.get(urlcomp, headers=headers)
    if "Please see the locations were cases have occurred:" in rcomp.text:
        print("Case numbers found")
        #print(rcomp.text)
        data_array[press_release_id]=[]
        soup = BeautifulSoup(rcomp.text,"lxml")
        html_content = soup.prettify()
        #print(html_content)
        for ultag in soup.find_all('strong'):
            #print(ultag.text)
            date_to_parse = ultag.text        
        
        for ultag in soup.find_all('ul'):
            #print(ultag.text)
            for litag in ultag.find_all('li'):
                returned_output=parse_list(litag.text)
                print(returned_output)
                if returned_output is not None:
                    #print(returned_output)
                    data_array[press_release_id]=returned_output  
                    remove_level = data_array[press_release_id]
                    print(remove_level)
                    df = pd.DataFrame.from_dict(remove_level)
                    df = df.assign(date = date_to_parse)
                    #df.to_parquet(f'../notebooks/p{press_release_id}.parquet')
        #loads = json.loads(json.dumps(data_array))
        #print(pd.DataFrame(loads[press_release_id]))
        return df

In [10]:
test_me = [2476, 2489]
for press_release_id in test_me:
    urlcomp="http://publichealth.lacounty.gov/phcommon/public/media/mediapubhpdetail.cfm?prid="+str(press_release_id)

    # Ignoring a duplicate spanish release
    df = get_data(urlcomp, press_release_id)
    print(df)
    if df is not None:
        df = (df.loc[: , [0,1, 'date']]
              .rename(columns = {0: 'region', 
                                1: 'cases'})
             )
        display(press_release_id)
        display(df.head(1))


None
None
